In [1]:
# Import Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

In [5]:
# Import data from WeatherPy folder
file = "../WeatherPy/Output/cities.csv"
weather_df = pd.read_csv(file)
df = pd.DataFrame(data)
df.head()

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (MPH)
0,morelia,MX,19.70,-101.18,69.80,68.0,75.0,3.36
1,ushuaia,AR,-54.80,-68.30,33.80,64.0,75.0,6.93
2,hilo,US,19.73,-155.09,82.40,58.0,75.0,11.41
3,hermanus,ZA,-34.42,19.23,50.00,96.0,6.0,1.99
4,chokurdakh,RU,70.63,147.92,55.99,40.0,100.0,9.22


In [8]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store Humidity and Latitude/Longitude
humidity = weather_df["Humidity (%)"]
locations = weather_df[["Latitude", "Longitude"]]

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=max(weather_df["Humidity (%)"]), point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Temperature (F)"] < 80) & (filtered_weather_df["Temperature (F)"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed (MPH)"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness (%)"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (MPH)
120,korla,CN,41.76,86.15,75.99,76.0,0.0,4.25
128,jalu,LY,29.03,21.55,73.89,45.0,0.0,5.77
129,morondava,MG,-20.28,44.28,70.32,52.0,0.0,9.24
181,vila vicosa,PT,38.78,-7.42,71.01,64.0,0.0,2.17
241,birjand,IR,32.87,59.22,73.40,16.0,0.0,6.93
277,sao jose da coroa grande,BR,-8.90,-35.15,75.07,83.0,0.0,6.22
279,asopos,GR,36.73,22.87,74.66,63.0,0.0,2.13
335,tripoli,LY,32.88,13.19,74.23,66.0,0.0,2.77
388,athens,GR,37.98,23.72,79.00,60.0,0.0,3.36
390,orihuela,ES,38.08,-0.94,73.00,91.0,0.0,4.88


In [10]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Data for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Data for Index 120: korla.
Missing field/result... skipping.
------------
Retrieving Data for Index 128: jalu.
Missing field/result... skipping.
------------
Retrieving Data for Index 129: morondava.
Missing field/result... skipping.
------------
Retrieving Data for Index 181: vila vicosa.
Missing field/result... skipping.
------------
Retrieving Data for Index 241: birjand.
Missing field/result... skipping.
------------
Retrieving Data for Index 277: sao jose da coroa grande.
Missing field/result... skipping.
------------
Retrieving Data for Index 279: asopos.
Missing field/result... skipping.
------------
Retrieving Data for Index 335: tripoli.
Missing field/result... skipping.
------------
Retrieving Data for Index 388: athens.
Missing field/result... skipping.
------------
Retrieving Data for Index 390: orihuela.
Missing field/result... skipping.
------------
Retrieving Data for Index 424: bejar.
Missing field/result... skipping.
------------
Retrieving Data for Index 43

In [11]:
hotel_df

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (MPH)
120,korla,CN,41.76,86.15,75.99,76.0,0.0,4.25
128,jalu,LY,29.03,21.55,73.89,45.0,0.0,5.77
129,morondava,MG,-20.28,44.28,70.32,52.0,0.0,9.24
181,vila vicosa,PT,38.78,-7.42,71.01,64.0,0.0,2.17
241,birjand,IR,32.87,59.22,73.40,16.0,0.0,6.93
277,sao jose da coroa grande,BR,-8.90,-35.15,75.07,83.0,0.0,6.22
279,asopos,GR,36.73,22.87,74.66,63.0,0.0,2.13
335,tripoli,LY,32.88,13.19,74.23,66.0,0.0,2.77
388,athens,GR,37.98,23.72,79.00,60.0,0.0,3.36
390,orihuela,ES,38.08,-0.94,73.00,91.0,0.0,4.88


In [12]:
# Assign the marker layer to a variable
locations = hotel_df[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

KeyError: 'Hotel Name'